In [1]:
import pandas as pd
import numpy as np
import re
df_profiles = pd.read_csv("student_or_staff_profiles.csv")
df_profiles['student_id/staff_id'] = df_profiles['student_id'].fillna(df_profiles['staff_id'])
df_profiles = df_profiles[['entity_id','name','role','email','department','student_id/staff_id','card_id','device_hash','face_id']]
df_swipes = pd.read_csv("campus_card_swipes.csv")
df_wifi = pd.read_csv("wifi_associations_logs.csv")
df_wifi['timestamp']=pd.to_datetime(df_wifi['timestamp'])


In [2]:
swipes_with_person = pd.merge(df_swipes[['card_id','location_id','timestamp']], df_profiles[['card_id', 'student_id/staff_id', 'name','role','entity_id','email','department','face_id']], on='card_id', how='left')
wifi_with_person = pd.merge(df_wifi[['device_hash','ap_id','timestamp',]], df_profiles[['device_hash', 'student_id/staff_id', 'name','role','entity_id','email','department','face_id']], on='device_hash', how='left')

In [3]:
swipes_std = swipes_with_person.copy()
swipes_std.rename(columns={'location_id': 'location_id/ap_id'}, inplace=True)
swipes_std.rename(columns={'card_id': 'card_id/device_hash'}, inplace=True)
swipes_std['event_type'] = 'card_swipe'

wifi_std = wifi_with_person.copy()
wifi_std.rename(columns={'ap_id': 'location_id/ap_id'}, inplace=True)
wifi_std.rename(columns={'device_hash': 'card_id/device_hash'}, inplace=True)
wifi_std['event_type'] = 'wifi_log'


In [4]:
unified_timeline_swipe_wifi = pd.concat([swipes_std, wifi_std], ignore_index=True)
unified_timeline_swipe_wifi.sort_values(by=['student_id/staff_id', 'timestamp'], inplace=True)
unified_timeline_swipe_wifi.reset_index(drop=True, inplace=True)
#unified_timeline_swipe_wifi

In [5]:
#unified_timeline_swipe_wifi.to_csv("unified_timeline_swipe_wifi.csv")

In [6]:
df_checkouts = pd.read_csv("library_checkouts.csv")

checkouts_with_person = pd.merge(df_checkouts, df_profiles[['card_id', 'student_id/staff_id', 'name','role','entity_id','email','department','face_id']], on = 'entity_id', how = 'left')
checkouts_with_person['checkout_id(book_id)'] = checkouts_with_person['checkout_id']+"("+checkouts_with_person['book_id']+")"

checkouts_std = checkouts_with_person[['checkout_id(book_id)','timestamp','student_id/staff_id','name','entity_id','role','email','department','face_id']]
checkouts_std['location_id/ap_id']='LIBRARY'
checkouts_std.rename(columns={'checkout_id(book_id)': 'checkout_id(book_id)/card_id/device_hash'}, inplace=True)
checkouts_std['event_type'] = 'library_checkout'
checkouts_std = checkouts_std[['checkout_id(book_id)/card_id/device_hash','location_id/ap_id','timestamp','student_id/staff_id','name','role','entity_id','event_type','email','department','face_id']]
#checkouts_std

C:\Users\nilot\AppData\Local\Temp\ipykernel_58492\1485214148.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  checkouts_std['location_id/ap_id']='LIBRARY'
C:\Users\nilot\AppData\Local\Temp\ipykernel_58492\1485214148.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  checkouts_std.rename(columns={'checkout_id(book_id)': 'checkout_id(book_id)/card_id/device_hash'}, inplace=True)
C:\Users\nilot\AppData\Local\Temp\ipykernel_58492\1485214148.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

In [7]:
unified_timeline_swipe_wifi.rename(columns={'card_id/device_hash': 'checkout_id(book_id)/card_id/device_hash'}, inplace=True)
unified_timeline_swipe_wifi_checkouts = pd.concat([unified_timeline_swipe_wifi, checkouts_std], ignore_index=True)
unified_timeline_swipe_wifi_checkouts.sort_values(by=['student_id/staff_id', 'timestamp'], inplace=True)
unified_timeline_swipe_wifi_checkouts.reset_index(drop=True, inplace=True)
unified_timeline_swipe_wifi_checkouts.rename(columns={'location_id/ap_id': 'location_id/ap_id/room_id'}, inplace=True)
#unified_timeline_swipe_wifi_checkouts

In [8]:
#unified_timeline_swipe_wifi_checkouts.to_csv("unified_timeline_swipe_wifi_checkouts.csv")

In [9]:
df_lab = pd.read_csv("lab_bookings.csv")
df_lab['start_time'] = pd.to_datetime(df_lab['start_time'])
df_lab['end_time'] = pd.to_datetime(df_lab['end_time'])
df_lab = df_lab[df_lab['attended (YES/NO)']=='YES']
df_lab.reset_index(drop=True, inplace=True)
df_lab = df_lab[['booking_id','entity_id','room_id','start_time','end_time']]
df_lab.rename(columns={'booking_id':'booking_id/checkout_id(book_id)/card_id/device_hash'}, inplace = True)

df_lab_start = df_lab[['booking_id/checkout_id(book_id)/card_id/device_hash','entity_id','room_id','start_time']]
df_lab_end = df_lab[['booking_id/checkout_id(book_id)/card_id/device_hash','entity_id','room_id','end_time']]

df_lab_start.rename(columns={'room_id':'location_id/ap_id/room_id'}, inplace = True)
df_lab_end.rename(columns={'room_id':'location_id/ap_id/room_id'}, inplace = True)

df_lab_start.rename(columns={'start_time':'timestamp'}, inplace = True)
df_lab_end.rename(columns={'end_time':'timestamp'}, inplace = True)

df_lab_start['event_type']='lab_start'
df_lab_end['event_type']='lab_end'

labs_with_person_start = pd.merge(df_lab_start, df_profiles, on = 'entity_id', how = 'left')
labs_with_person_end = pd.merge(df_lab_end, df_profiles, on = 'entity_id', how = 'left')

labs_with_person_start_std = labs_with_person_start[['booking_id/checkout_id(book_id)/card_id/device_hash','location_id/ap_id/room_id','timestamp','student_id/staff_id','name','role','entity_id','event_type','email','department','face_id']]
labs_with_person_end_std = labs_with_person_end[['booking_id/checkout_id(book_id)/card_id/device_hash','location_id/ap_id/room_id','timestamp','student_id/staff_id','name','role','entity_id','event_type','email','department','face_id']]

labs_with_person_std = pd.concat([labs_with_person_start_std, labs_with_person_end_std], ignore_index=True)
#labs_with_person_std

C:\Users\nilot\AppData\Local\Temp\ipykernel_58492\101524182.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lab_start.rename(columns={'room_id':'location_id/ap_id/room_id'}, inplace = True)
C:\Users\nilot\AppData\Local\Temp\ipykernel_58492\101524182.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lab_end.rename(columns={'room_id':'location_id/ap_id/room_id'}, inplace = True)
C:\Users\nilot\AppData\Local\Temp\ipykernel_58492\101524182.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-do

In [10]:
unified_timeline_swipe_wifi_checkouts.rename(columns={'checkout_id(book_id)/card_id/device_hash': 'booking_id/checkout_id(book_id)/card_id/device_hash'}, inplace=True)
unified_timeline_swipe_wifi_checkouts_labs = pd.concat([unified_timeline_swipe_wifi_checkouts, labs_with_person_std ], ignore_index=True)
unified_timeline_swipe_wifi_checkouts_labs.sort_values(by=['student_id/staff_id', 'timestamp'], inplace=True)
unified_timeline_swipe_wifi_checkouts_labs.reset_index(drop=True, inplace=True)
unified_timeline_swipe_wifi_checkouts_labs = unified_timeline_swipe_wifi_checkouts_labs[['student_id/staff_id','timestamp','name','role','department','email','booking_id/checkout_id(book_id)/card_id/device_hash','location_id/ap_id/room_id','entity_id','face_id','event_type']]
unified_timeline_swipe_wifi_checkouts_labs

,student_id/staff_id,timestamp,name,role,department,email,booking_id/checkout_id(book_id)/card_id/device_hash,location_id/ap_id/room_id,entity_id,face_id,event_type
0,S10011,2025-09-10 23:31:00,Siddharth Gupta,student,Admin,user4043@campus.edu,BKG305120,LAB_101,E104043,F104043,lab_start
1,S10011,2025-09-11 01:14:00,Siddharth Gupta,student,Admin,user4043@campus.edu,BKG305120,LAB_101,E104043,F104043,lab_end
2,S10011,2025-08-29 11:37:00,Siddharth Gupta,student,Admin,user4043@campus.edu,LC206020(BK2525),LIBRARY,E104043,F104043,library_checkout
3,S10011,2025-09-01 21:08:28,Siddharth Gupta,student,Admin,user4043@campus.edu,C5438,ADMIN_LOBBY,E104043,F104043,card_swipe
4,S10011,2025-09-07 08:42:24,Siddharth Gupta,student,Admin,user4043@campus.edu,C5438,LIB_ENT,E104043,F104043,card_swipe
...,...,...,...,...,...,...,...,...,...,...,...
42816,T9995,2025-09-06 14:43:46,Aarav Rao,staff,Physics,user5398@campus.edu,C1868,LAB_101,E105398,NaN,card_swipe
42817,T9995,2025-09-09 19:20:44,Aarav Rao,staff,Physics,user5398@campus.edu,C1868,LIB_ENT,E105398,NaN,card_swipe
42818,T9995,2025-09-10 12:18:52,Aarav Rao,staff,Physics,user5398@campus.edu,C1868,LAB_101,E105398,NaN,card_swipe
42819,T9995,2025-09-22 01:21:06,Aarav Rao,staff,Physics,user5398@campus.edu,LC200741(BK2814),LIBRARY,E105398,NaN,library_checkout


In [11]:
unified_timeline_swipe_wifi_checkouts_labs.to_csv("unified_timeline_swipe_wifi_checkouts_labs.csv")

In [12]:
df_notes = pd.read_csv("free_text_notes (helpdesk or RSVPs).csv")
pattern = r'(in|near|requesting|for)\s+([A-Z_0-9]+)'
df_notes['extracted_location'] = df_notes['text'].str.extract(pattern, flags=re.IGNORECASE)[1]
condition = df_notes['extracted_location'] == 'seminar'
df_notes.loc[condition, 'extracted_location'] = 'seminar room'
dondition = df_notes['extracted_location'] == 'hostel'
df_notes.loc[dondition, 'extracted_location'] = 'hostel block'
eondition = df_notes['extracted_location'] == 'robotics'
df_notes.loc[eondition, 'extracted_location'] = 'robotics workshop'
df_notes

,note_id,entity_id,category,text,timestamp,extracted_location
0,N400000,E101498,rsvp,Confirmed attendance for robotics workshop.,2025-08-30 02:51:39,robotics workshop
1,N400001,E101738,helpdesk,Wi-Fi not working in hostel block.,2025-09-18 03:03:05,hostel block
2,N400002,E101590,maintenance,CCTV near library needs check.,2025-09-06 19:47:08,library
3,N400003,E101144,feedback,Broken chair in seminar room.,2025-09-19 09:05:20,seminar room
4,N400004,E102891,incident,Wi-Fi not working in hostel block.,2025-09-11 13:28:20,hostel block
...,...,...,...,...,...,...
6995,N406995,E100481,maintenance,CCTV near library needs check.,2025-09-25 21:29:27,library
6996,N406996,E100278,feedback,Wi-Fi not working in hostel block.,2025-09-19 22:09:23,hostel block
6997,N406997,E105224,feedback,Confirmed attendance for robotics workshop.,2025-09-17 22:24:48,robotics workshop
6998,N406998,E101619,maintenance,CCTV near library needs check.,2025-09-12 06:42:02,library


In [13]:
# df_notes.to_csv("CLEANED~free_text_notes.csv")

In [14]:
notes_with_person = pd.merge(df_notes, df_profiles, on = 'entity_id', how = 'left')
notes_with_person.rename(columns={'note_id': 'booking_id/checkout_id(book_id)/card_id/device_hash/note_id'}, inplace=True)
notes_with_person.rename(columns={'extracted_location': 'location/location_id/ap_id/room_id'}, inplace=True)
notes_with_person['category'] = 'note' + '(' +  notes_with_person['category'] + ')' + ':' + notes_with_person['text']
notes_with_person.rename(columns={'category': 'event_type'}, inplace=True)
notes_std = notes_with_person[['student_id/staff_id','timestamp','name','role','department','email','booking_id/checkout_id(book_id)/card_id/device_hash/note_id','location/location_id/ap_id/room_id','entity_id','face_id','event_type']]
notes_std

,student_id/staff_id,timestamp,name,role,department,email,booking_id/checkout_id(book_id)/card_id/device_hash/note_id,location/location_id/ap_id/room_id,entity_id,face_id,event_type
0,S41922,2025-08-30 02:51:39,Sana Rao,student,Physics,user1498@campus.edu,N400000,robotics workshop,E101498,F101498,note(rsvp):Confirmed attendance for robotics w...
1,S89215,2025-09-18 03:03:05,Karan Patel,student,BIO,user1738@campus.edu,N400001,hostel block,E101738,F101738,note(helpdesk):Wi-Fi not working in hostel block.
2,S96850,2025-09-06 19:47:08,Sana Kumar,student,ECE,user1590@campus.edu,N400002,library,E101590,F101590,note(maintenance):CCTV near library needs check.
3,S92312,2025-09-19 09:05:20,Sana Gupta,student,EEE,user1144@campus.edu,N400003,seminar room,E101144,F101144,note(feedback):Broken chair in seminar room.
4,T6127,2025-09-11 13:28:20,Karan Rao,staff,Physics,user2891@campus.edu,N400004,hostel block,E102891,F102891,note(incident):Wi-Fi not working in hostel block.
...,...,...,...,...,...,...,...,...,...,...,...
6995,S52898,2025-09-25 21:29:27,Siddharth Patel,student,Chemistry,user481@campus.edu,N406995,library,E100481,F100481,note(maintenance):CCTV near library needs check.
6996,T2299,2025-09-19 22:09:23,Ishaan Desai,faculty,EEE,user278@campus.edu,N406996,hostel block,E100278,F100278,note(feedback):Wi-Fi not working in hostel block.
6997,S59620,2025-09-17 22:24:48,Karan Gupta,student,Maths,user5224@campus.edu,N406997,robotics workshop,E105224,NaN,note(feedback):Confirmed attendance for roboti...
6998,T4450,2025-09-12 06:42:02,Ananya Singh,staff,EEE,user1619@campus.edu,N406998,library,E101619,F101619,note(maintenance):CCTV near library needs check.


In [15]:
unified_timeline_swipe_wifi_checkouts_labs.rename(columns={'booking_id/checkout_id(book_id)/card_id/device_hash': 'booking_id/checkout_id(book_id)/card_id/device_hash/note_id'}, inplace=True)
unified_timeline_swipe_wifi_checkouts_labs.rename(columns={'location_id/ap_id/room_id': 'location/location_id/ap_id/room_id'}, inplace=True)
unified_timeline_swipe_wifi_checkouts_labs_notes = pd.concat([unified_timeline_swipe_wifi_checkouts_labs, notes_std ], ignore_index=True)
unified_timeline_swipe_wifi_checkouts_labs_notes.sort_values(by=['student_id/staff_id', 'timestamp'], inplace=True)
unified_timeline_swipe_wifi_checkouts_labs_notes.reset_index(drop=True, inplace=True)
unified_timeline_swipe_wifi_checkouts_labs_notes

,student_id/staff_id,timestamp,name,role,department,email,booking_id/checkout_id(book_id)/card_id/device_hash/note_id,location/location_id/ap_id/room_id,entity_id,face_id,event_type
0,S10011,2025-09-10 23:31:00,Siddharth Gupta,student,Admin,user4043@campus.edu,BKG305120,LAB_101,E104043,F104043,lab_start
1,S10011,2025-09-11 01:14:00,Siddharth Gupta,student,Admin,user4043@campus.edu,BKG305120,LAB_101,E104043,F104043,lab_end
2,S10011,2025-08-29 11:37:00,Siddharth Gupta,student,Admin,user4043@campus.edu,LC206020(BK2525),LIBRARY,E104043,F104043,library_checkout
3,S10011,2025-09-01 21:08:28,Siddharth Gupta,student,Admin,user4043@campus.edu,C5438,ADMIN_LOBBY,E104043,F104043,card_swipe
4,S10011,2025-09-02 17:01:01,Siddharth Gupta,student,Admin,user4043@campus.edu,N405824,hostel block,E104043,F104043,note(rsvp):Wi-Fi not working in hostel block.
...,...,...,...,...,...,...,...,...,...,...,...
49816,T9995,2025-09-06 14:43:46,Aarav Rao,staff,Physics,user5398@campus.edu,C1868,LAB_101,E105398,NaN,card_swipe
49817,T9995,2025-09-09 19:20:44,Aarav Rao,staff,Physics,user5398@campus.edu,C1868,LIB_ENT,E105398,NaN,card_swipe
49818,T9995,2025-09-10 12:18:52,Aarav Rao,staff,Physics,user5398@campus.edu,C1868,LAB_101,E105398,NaN,card_swipe
49819,T9995,2025-09-22 01:21:06,Aarav Rao,staff,Physics,user5398@campus.edu,LC200741(BK2814),LIBRARY,E105398,NaN,library_checkout


In [16]:
unified_timeline_swipe_wifi_checkouts_labs_notes.to_csv("unified_timeline_swipe_wifi_checkouts_labs_notes.csv")